In [1]:
import parser
import numpy as np
import gzip, msgpack
from torch.nn.modules import PairwiseDistance
import random
import torch 
import torch.nn as nn
from torch.autograd import Variable
import scipy.sparse as sp
import time

In [2]:
from sklearn.feature_extraction.text import HashingVectorizer
N_FEATURES = 2**18
hv = HashingVectorizer(n_features=N_FEATURES,non_negative=True, stop_words='english')
# try default n-gram 1-gram,2-gram,3-gram

In [3]:
class SmallReference:
    def __init__(self, identifiers):
        self.identifiers = identifiers

class SmallPaper:
    def __init__(self, title, identifiers, authors, abstract, refs):
        self.title = title
        self.authors = authors
        self.abstract = hv.transform([abstract])
        self.references = [SmallReference(ref.identifiers) for ref in refs]
        self.identifiers = identifiers

In [4]:
def get_doi(paper):
    _ref_dict = dict()
    for x in paper.identifiers:
#        @todo lowecase
        _ref_dict[x.key_type] = x.key.lower()
    if 'doi' in _ref_dict:
        return _ref_dict['doi']
    elif 'ieee' in _ref_dict:
        return _ref_dict['ieee']
    elif 'semanticscholar' in _ref_dict:
        return _ref_dict['semanticscholar']
    elif 'adsa' in _ref_dict:
        return _ref_dict['adsa']
    else:
        return False

    
def update_authors(ident_to_auth, authors_to_papers, paper):
    for author in paper.authors:
        author_name = author.name.name
        
        if not author_name:
            continue
        for identifier in author.identifiers:
            result = ident_to_auth.get(identifier.key.lower(), False) 
            if result:
                author_name = str(result)
                break
        for identifier in author.identifiers:
            ident_to_auth[identifier.key.lower()] = author_name
        if not authors_to_papers.get(author_name, False):
            authors_to_papers[author_name] = list()
        authors_to_papers[author_name].append(get_doi(paper))

In [5]:


identies_set = dict()

_paper_dict = dict()
myiter = 0
count=0
_ref_dict = dict()
_training_set = list()
_validation_set = list()
_test_set = list()
_ident_to_auth_dict = dict()
_authors_to_papers_dict = dict()
with gzip.open("NN_Papers.msgpack.gz", "rb") as nn_papers_out:
    unpacker = msgpack.Unpacker(nn_papers_out, encoding='utf-8')
    for _paper in unpacker:
        count+=1
        paper = parser.Paper.deserialize(_paper)
        
        paper_doi = get_doi(paper)
        if not (paper_doi and paper.abstract):
            continue
        else:
            myiter+=1
        paper = SmallPaper(paper.title, paper.identifiers, paper.authors, paper.abstract, paper.references)
        if myiter>=10000:
            break
        #calculate statistics
        for iden in paper.identifiers:
            if identies_set.get(iden.key_type, False):
                identies_set[iden.key_type]+=1
            else:
                identies_set[iden.key_type]=1
        #end
        
        _ref_dict[paper_doi] = list()
 
        update_authors(_ident_to_auth_dict, _authors_to_papers_dict, paper)
        _paper_dict[paper_doi] = paper
        if myiter<=160000:
            _training_set.append(paper_doi)
        elif myiter<=180000: 
            _test_set.append(paper_doi)
        else:
            _validation_set.append(paper_doi)

print('total count: {0}'.format(count))
print(identies_set)

total count: 14456
{'adsa': 2260, 'ieee': 3468, 'iop': 189, 'onepetro': 45, 'arxiv': 520, 'pmc': 361, 'acm': 1904, 'doi': 7638, 'oxford': 211, 'mid': 23, 'semanticscholar': 2702, 'science': 5, 'springer': 856, 'sage': 38, 'pmid': 1310, 'nature': 105, 'manuscript': 47}


In [6]:
def update_references(paper_dict, ref_dict):
    count = 0
    for paper_doi,paper in paper_dict.items():
        for ref in paper.references:
            ref_doi = get_doi(ref)
            if not ref_doi:
                continue
            result = ref_dict.get(ref_doi, False)
            if result!=False:
                count+=1
                ref_dict[paper_doi].append(ref_doi)
    print(count)
update_references(_paper_dict, _ref_dict)

1033


In [7]:
# list(_paper_.keys())[6]
# get_doi(_paper_dict['10.1109/tnnls.2015.2392563'].references[4])

In [8]:
# abstract_training_set = [paper.abstract for paper in training_set]
# abstract_training_set_hashed = hv.transform(abstract_training_set)

In [9]:
dtype = torch.FloatTensor

def make_var(abstract):
    dense = abstract.todense()
    torch_tensor = Variable(torch.from_numpy(dense).type(dtype))
    return torch_tensor

In [ ]:

# random.randint(0,100)

In [ ]:
# identies_set = dict()
# for paper in paper_list.values():
#     for iden in paper.identifiers:
#         if identies_set.get(iden.key_type, False):
#             identies_set[iden.key_type]+=1
#         else:
#             identies_set[iden.key_type]=1
# print(identies_set)
# len(paper_list)

In [ ]:



# def get_triple(val_set):
#     count = 0
#     t0 = 0
#     t1 =0 b
#     for item in val_set:
#         count += 1
#         _ref_doi = get_doi(item)
#         if _ref_doi not in ref_dict.keys():
#             continue
#         _ref_list = ref_dict[_ref_doi]
        
#         for _ref in _ref_list:
#             if paper_list.get(_ref, False) in val_set:
#                 _neg_example = None
#                 while(not _neg_example):
#                     _neg_id = random.randint(0,len(val_set)-1)
#                     _neg_doi = get_doi(val_set[_neg_id])
#                     if (_neg_doi != _ref_doi) and (_neg_doi not in _ref_list):
#                         neg_ref_dict = ref_dict[_neg_doi]
#                         if not (_ref_doi in neg_ref_dict or _ref in neg_ref_dict):
#                             _neg_example = val_set[_neg_id]
# #                 t1 = time.time()
# #                 print('time to sample triple: {0}'.format(t1-t0))
#                 yield (item.abstract, paper_list[_ref].abstract, _neg_example.abstract)
# #                 t0 = time.time()

In [10]:
# appr = _paper_dict['10.1109/tnnls.2015.2392563']
def get_papers_per_authors(paper):
    set_to_merge = set()
    for author in paper.authors:
        identifier = list(author.identifiers)[0].key
#         print("name {0}. papers: {1}".format(_ident_to_auth_dict[identifier],
#                                              _authors_to_papers_dict[_ident_to_auth_dict[identifier]]))
        set_to_merge.update(set(_authors_to_papers_dict[_ident_to_auth_dict[identifier.lower()]]))
    set_to_merge.discard(get_doi(paper))
    return set_to_merge

In [ ]:
# get_papers_per_authors(_paper_dict['10.1109/tnnls.2015.2392563'])
# [x for x in (1,2,3,4)]
{1,3,4,5}.intersection([4,5])

In [11]:
BATCH_SIZE = 100

def prepare_triples(val_set):
    count = 0
    result_list = list()
    batch_list_1 = None
    batch_list_2 = None
    batch_list_3 = None
#     print(len(val_set))
    for paper_doi in val_set:
        paper = _paper_dict[paper_doi]
        
        
        pos_ref_set = get_papers_per_authors(paper).intersection(val_set)
        
        pos_ref_set.update(set(_ref_dict[paper_doi]).intersection(val_set))
        #cut down articles that are not used in val_set
        
#         pos_ref_set.intersection(val_set)
        
        if len(pos_ref_set) == 0:
#             print(paper_doi)
            continue
#         print(len(pos_ref_set))
        neg_ref_set = set()
        for n in range(len(pos_ref_set)):
            new_id = None
            while((new_id in pos_ref_set) or new_id==None):
                new_id = val_set[random.randrange(0,len(val_set)-1)]
            neg_ref_set.add(new_id)
            
            
        for pos,neg in zip(pos_ref_set, neg_ref_set):

            if batch_list_1 == None:
                batch_list_1 = paper.abstract
                batch_list_2 = _paper_dict[pos].abstract
                batch_list_3 = _paper_dict[neg].abstract
            else:
                batch_list_1 = sp.vstack((batch_list_1, paper.abstract))
                batch_list_2 = sp.vstack((batch_list_2, _paper_dict[pos].abstract))
                batch_list_3 = sp.vstack((batch_list_3, _paper_dict[neg].abstract))

            if batch_list_1.shape[0]==BATCH_SIZE:
                yield (batch_list_1, batch_list_2, batch_list_3)
                batch_list_1 = None
#     print(count)
#     return result_list


    

In [ ]:

# get_papers_per_authors(_paper_dict[_training_set[2]]).intersection(_training_set[:50])

In [ ]:
# a1,a2,a3 = next(triples_iterator)

In [ ]:
# a1[0]

In [ ]:



# dtype = torch.FloatTensor
# def get_batch(iterator, batch_size=BATCH_SIZE):
#     a1 = None
#     a2 = None
#     a3 = None
#     for x1,x2,x3 in iterator:
#         if a1 == None:
#             a1=x1
#             a2=x2
#             a3=x3
#         else:
#             a1 = sp.vstack((a1,x1))
#             a2 = sp.vstack((a2,x2))
#             a3 = sp.vstack((a3,x3))            
#         if a1.shape[0]==batch_size:
#             dense_a1 = a1.todense()
#             dense_a2 = a2.todense()
#             dense_a3 = a3.todense()
#             tor_a1 = torch.from_numpy(dense_a1).type(dtype)
#             tor_a2 = torch.from_numpy(dense_a2).type(dtype)
#             tor_a3 = torch.from_numpy(dense_a3).type(dtype)
#             yield (Variable(tor_a1),
#                    Variable(tor_a2),
#                    Variable(tor_a3))
#             a1 = None
#             a2 = None
#             a3 = None


# input data sparse matrix


In [ ]:
hv.transform(['avasd', 'asdfwe sdfasdfwe werwe adsfsd']).todense()

In [ ]:
class DistRes2(nn.Module):

    def __init__(self):
        super(DistRes, self).__init__()
        self.layer1 = nn.Linear(N_FEATURES, 100)
        self.layer2 = nn.PairwiseDistance(2)

    def forward(self, x1,x2):
#         print('typex: {0}'.format(type(x1)))
#         print('typey: {0}'.format(type(x2)))
        res1 = self.layer1(x1)
        res2 = self.layer1(x2)
        # normalize vector L2-normalization
        return self.layer2(res1,res2)
    

In [47]:
class DistRes(nn.Module):

    def __init__(self):
        super(DistRes, self).__init__()
        self.layer1 = nn.Linear(N_FEATURES, 100)
        self.layer2 = nn.PairwiseDistance(2)

    def forward(self, x1,x2):
#         print('typex: {0}'.format(type(x1)))
#         print('typey: {0}'.format(type(x2)))
        res1 = self.layer1(x1)
        res2 = self.layer1(x2)
        # normalize vector L2-normalization
        return self.layer2(res1,res2)
    
    def calculate_res(self, x):
        return self.layer1(x).data.numpy()
    
# linear/RelU/linear
# dropout input layer
# help as they authored by same person

In [48]:

dist_model = DistRes()

In [49]:
#check the distance Loss, just to see the difference
# maybe pos and pos close

crit = nn.HingeEmbeddingLoss(1)
optim = torch.optim.SGD(dist_model.parameters(), lr=1e-1)
# local_dist(z)

In [50]:
ones_var = Variable(torch.ones(BATCH_SIZE))
neg_var = Variable(torch.ones(BATCH_SIZE)*(-1))

def train_networks(model, criteria, optimizer, paper, p_pos, p_neg):
                    
    loss_pos = criteria.forward( model(paper, p_pos), ones_var)
    optimizer.zero_grad()
    gradCrit = loss_pos.backward()
    optimizer.step()

    loss_neg = criteria.forward( model(paper,p_neg), neg_var)
    optimizer.zero_grad()
    gradCrit = loss_neg.backward()
    optimizer.step()
    
    return model(paper, p_pos), model(paper,p_neg)




In [51]:
import time


In [90]:
epochs = 10
for i in range(epochs):
    aver_pos = list()
    aver_neg = list()
    
    batch=0
    t0=0
    t1=0
    triples_iterator = prepare_triples(_training_set[:3000])
    
    for arg1,arg2,arg3 in triples_iterator:
        t0 = time.time()
#         print('sampling time: {0}'.format(t0-t1))
        ap, an = train_networks(model=dist_model, criteria=crit, optimizer=optim, 
                                paper=make_var(arg1), p_pos=make_var(arg2), 
                                p_neg=make_var(arg3))
        t1 = time.time()
        total= t1-t0
        aver_pos.append(ap.data.mean())
        aver_neg.append(an.data.mean())
        print('Epoch: {0}. Batch: {3}. Time {4} av_pos: {1}. av_neg: {2}'.format(i, np.mean(aver_pos), 
                                                                                 np.mean(aver_neg), 
                                                                                 batch, total))
        batch+=1
    
    #calc auc
    auc = calculate_AUC(model=dist_model, val_set=_training_set[3000:4000])
    
    print('Epoch: {0}. av_pos: {1}. av_neg: {2}. AUC: {3}'.format(i, np.mean(aver_pos), np.mean(aver_neg), auc))
    


Epoch: 0. Batch: 0. Time 1.2754223346710205 av_pos: 0.02194072071619303. av_neg: 0.0821487883105874
Epoch: 0. Batch: 1. Time 1.3558611869812012 av_pos: 0.022787900837047348. av_neg: 0.08486065577715635
Epoch: 0. Batch: 2. Time 1.3750531673431396 av_pos: 0.023493933553824415. av_neg: 0.08452777655795217
Epoch: 0. Batch: 3. Time 1.3301098346710205 av_pos: 0.023863562273918434. av_neg: 0.08382976029533892
Epoch: 0. Batch: 4. Time 1.31687331199646 av_pos: 0.024379226570919857. av_neg: 0.08241948178783058
Epoch: 0. Batch: 5. Time 1.3350005149841309 av_pos: 0.023964836281678194. av_neg: 0.08224251994552711
Epoch: 0. Batch: 6. Time 1.3945679664611816 av_pos: 0.02614211866248037. av_neg: 0.0814134466914194
Epoch: 0. Batch: 7. Time 1.35892915725708 av_pos: 0.02875351283325017. av_neg: 0.079891101969406
Epoch: 0. Batch: 8. Time 1.24531888961792 av_pos: 0.03120878618291802. av_neg: 0.0788178363463117
Epoch: 0. av_pos: 0.03120878618291802. av_neg: 0.0788178363463117. AUC: 0.5625
Epoch: 1. Batch: 0

Epoch: 8. Batch: 3. Time 1.3771741390228271 av_pos: 0.03588277766038118. av_neg: 0.14243358528707176
Epoch: 8. Batch: 4. Time 1.35357666015625 av_pos: 0.036961037310567914. av_neg: 0.14002531630918383
Epoch: 8. Batch: 5. Time 1.332549810409546 av_pos: 0.03630315011014317. av_neg: 0.14107720802758195
Epoch: 8. Batch: 6. Time 1.339008092880249 av_pos: 0.03891681116226599. av_neg: 0.1396719622710693
Epoch: 8. Batch: 7. Time 1.3642454147338867 av_pos: 0.04102227429488721. av_neg: 0.13757943477646678
Epoch: 8. Batch: 8. Time 1.3498926162719727 av_pos: 0.04327309034331847. av_neg: 0.13574253937583208
Epoch: 8. av_pos: 0.04327309034331847. av_neg: 0.13574253937583208. AUC: 0.5892857142857143
Epoch: 9. Batch: 0. Time 1.289754867553711 av_pos: 0.03535583787928772. av_neg: 0.14618756148964165
Epoch: 9. Batch: 1. Time 1.2452471256256104 av_pos: 0.035646155736958465. av_neg: 0.1534706825762987
Epoch: 9. Batch: 2. Time 1.3615360260009766 av_pos: 0.036678173833194404. av_neg: 0.15263691001882154
Epo

In [ ]:
triple_iterator1 = get_triple(training_set)
batch_iterator1 = get_batch(triple_iterator1)
arg1,arg2,arg3 = next(batch_iterator1)
k = dist_model(arg1,arg2)

In [89]:

RECALL_N = 10

def calculate_AUC(model, val_set):
    neg_examples_amount = min(100, int(len(val_set)/10))
    a=0
    recall_at_n = list()
    
    for paper_doi in val_set:
        paper = _paper_dict[paper_doi]
        pos_ref_set = get_papers_per_authors(paper).intersection(val_set)
        pos_ref_set.update(set(_ref_dict[paper_doi]).intersection(val_set))
        if len(pos_ref_set)==0:
            continue
        
        neg_ref_set = set()
        for n in range(neg_examples_amount):
            new_id = None
            while((new_id in pos_ref_set) or new_id==None):
                new_id = val_set[random.randrange(0,neg_examples_amount)]
            neg_ref_set.add(new_id)
        
        all_refs = set()
        all_refs.update(pos_ref_set)
        all_refs.update(neg_ref_set)
        
        paper_vector = model.calculate_res(make_var(paper.abstract))
        
        #calculate
        distance_array = dict()
        for ref in all_refs: 
            ref_vec = model.calculate_res(make_var(_paper_dict[ref].abstract))
            ref_dist = np.linalg.norm(paper_vector - ref_vec)
            distance_array[ref] = ref_dist

        ordered_predicted_list = list(map(lambda x: x[0], 
                                          sorted(distance_array.items(), 
                                                 key=lambda val: val[1])))[:RECALL_N]
        recall = len(pos_ref_set.intersection(ordered_predicted_list))/len(pos_ref_set)
        if a!=0:
            print(ordered_predicted_list)
            print(pos_ref_set)
            a-=1
        #print(recall)
        recall_at_n.append(recall)

    return np.mean(recall_at_n)
        
            
    

In [60]:
a = {'a':1,'b':5, 'c':3,'d':0}
b = list(map(lambda x: x[0], sorted(a.items(), key=lambda val: val[1])[:3]))
c = {'d', 'a'}
d = len(c.intersection(b))/len(c)
np.mean([d,0])
int(10 / 3)

3

In [ ]:
stacked_matrix.shape